## Tennessee Eastman（TE/TEP）プロセスのサンプルデータ

変数多いが手間がかかる

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi%3A10.7910%2FDVN%2F6C3JR1&utm_source=chatgpt.com

In [ ]:
import rdata
import pandas as pd
from pathlib import Path

# ============================================================
# TE（Harvard Dataverse版: .RData）読み込み（画像のコード準拠）
# ============================================================

DATA_DIR = Path(r".\TEPdata")  # 例: r"C:\...\TEP_harvard_raw"
FF_PATH  = DATA_DIR / "TEP_FaultFree_Testing.RData"
FY_PATH  = DATA_DIR / "TEP_Faulty_Testing.RData"

# ----------------------------
# 1) RData を読み込み → dictへ変換
# ----------------------------
ff_parsed = rdata.parser.parse_file(str(FF_PATH))
ff_conv   = rdata.conversion.convert(ff_parsed)

fy_parsed = rdata.parser.parse_file(str(FY_PATH))
fy_conv   = rdata.conversion.convert(fy_parsed)

# キー確認（想定: 'fault_free_testing', 'faulty_testing'）
print("FaultFree keys:", list(ff_conv.keys()))
print("Faulty   keys:", list(fy_conv.keys()))

if "fault_free_testing" not in ff_conv:
    raise KeyError(f"'fault_free_testing' が見つかりません: {list(ff_conv.keys())}")
if "faulty_testing" not in fy_conv:
    raise KeyError(f"'faulty_testing' が見つかりません: {list(fy_conv.keys())}")

# ----------------------------
# 2) DataFrame化（画像のコードと同じ）
# ----------------------------
ff_df = pd.DataFrame(ff_conv["fault_free_testing"])
fy_df = pd.DataFrame(fy_conv["faulty_testing"])

# faultNumber列名の揺れ対策（faultNumber / faultnumber 等）
fault_col = next((c for c in fy_df.columns if c.lower() == "faultnumber"), None)
if fault_col is None:
    raise KeyError(f"'faultNumber' 列が見つかりません: {list(fy_df.columns)[:30]} ...")

# ----------------------------
# 3) 画像のスライス規則で「正常」「チューニング」「故障ごと」を作る
#    ・行: 0:960 を1本ぶんの長さとして利用
#    ・列: 3: で先頭3列（メタ列）を落とす
# ----------------------------
normal_data = ff_df.iloc[0:960, 3:].reset_index(drop=True)       # 学習用データ（正常）
tuning_data = ff_df.iloc[960:1920, 3:].reset_index(drop=True)    # チューニング用（正常）

# 故障データ（faultNumber==i を取り、先頭960行・列3: だけ）
fault_data_dict = {}
for i in range(1, 21):  # 画像のコード準拠（1..20）
    idv = fy_df[fy_df[fault_col].astype(int) == i].iloc[0:960, 3:].reset_index(drop=True)
    fault_data_dict[i] = idv

print("normal_data:", normal_data.shape)
print("tuning_data:", tuning_data.shape)
print("fault 14    :", fault_data_dict[14].shape)

# ----------------------------
# 4) CSVに保存（必要なら：画像のコード準拠）
# ----------------------------
OUT_DIR = DATA_DIR / "csv_out"
OUT_DIR.mkdir(parents=True, exist_ok=True)

normal_data.to_csv(OUT_DIR / "normal-data.csv", index=False)
tuning_data.to_csv(OUT_DIR / "tuning-data.csv", index=False)

for i, df_i in fault_data_dict.items():
    df_i.to_csv(OUT_DIR / f"idv_str{i}_data.csv", index=False)
